In [1]:
from sklearn.datasets import load_boston
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import syft as sy
from syft.core.node.vm.vm import VirtualMachine
from syft.core.node.vm.client import VirtualMachineClient
from syft.ast.module import Module
from syft.core.remote_dataloader import RemoteDataLoader
from syft.core.remote_dataloader import RemoteDataset

print(f'torch version: {torch.__version__}')
print(f'syft version: {sy.__version__}')

torch version: 1.8.1+cu102
syft version: 0.5.0


# Constructing the client and server

In [2]:
server: VirtualMachine = sy.VirtualMachine(name="server")
client: VirtualMachineClient = server.get_root_client()

In [3]:
remote_torch: Module = client.torch
remote_torch

Module:
	.Tensor -> <syft.ast.klass.Class object at 0x7f8c668620a0>
	.BFloat16Tensor -> <syft.ast.klass.Class object at 0x7f8c66862100>
	.BoolTensor -> <syft.ast.klass.Class object at 0x7f8c66862160>
	.ByteTensor -> <syft.ast.klass.Class object at 0x7f8c668621c0>
	.CharTensor -> <syft.ast.klass.Class object at 0x7f8c66862220>
	.DoubleTensor -> <syft.ast.klass.Class object at 0x7f8c66862280>
	.FloatTensor -> <syft.ast.klass.Class object at 0x7f8c668622e0>
	.HalfTensor -> <syft.ast.klass.Class object at 0x7f8c66862340>
	.IntTensor -> <syft.ast.klass.Class object at 0x7f8c668623a0>
	.LongTensor -> <syft.ast.klass.Class object at 0x7f8c66862400>
	.ShortTensor -> <syft.ast.klass.Class object at 0x7f8c66862460>
	.nn -> Module:
		.Parameter -> <syft.ast.klass.Class object at 0x7f8c66862580>
		.Module -> <syft.ast.klass.Class object at 0x7f8be1a30880>
		.Conv2d -> <syft.ast.klass.Class object at 0x7f8be1a30d60>
		.Dropout2d -> <syft.ast.klass.Class object at 0x7f8be1a351c0>
		.Linear -> <syft.

# Client: loading and exploring the dataset

In [4]:
boston_dataset = load_boston()
data = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [5]:
target = pd.DataFrame(boston_dataset['target'])
target

,0
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


In [6]:
X = torch.tensor(boston_dataset["data"]).float()
y = torch.tensor(boston_dataset["target"]).float()

Let's split the dataset into a train dataset and a validation dataset (80%-20%)

In [18]:
train_val_ratio = 0.8
split_idx = int(len(X)*train_val_ratio)

In [19]:
X_train = X[:split_idx, :]
X_val = X[split_idx:, :]
y_train = y[:split_idx]
y_val = y[split_idx:]

TypeError: 'AnyPointer' object is not subscriptable

In [11]:
len(X_val) + len(X_train)

506

The client load the dataset locally and save it to `.pt` files

In [12]:
class BostonDS(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, i):
        return (self.X[i], self.y[i])
    
ds_train = BostonDS(X_train, y_train)
ds_val = BostonDS(X_val, y_val)
torch.save(ds_train, "boston_train_ds.pt")
torch.save(ds_val, "boston_val_ds.pt")

The client would then send this string path to the server

# Server: create the remote dataset and remote dataloader

In [13]:
boston_train_rds = RemoteDataset(path='boston_train_ds.pt', data_type="torch_tensor")
boston_train_rds

<class 'syft.core.remote_dataloader.remote_dataloader.RemoteDataset'>: torch_tensor

From the remote dataset, the server constructs the data loader. 
Then the server uses `.send` to create a pointer to do remote data loading

In [14]:
boston_train_rdl = RemoteDataLoader(remote_dataset=boston_train_rds, batch_size=16)
boston_train_rdl

In [15]:
boston_train_ptr = boston_train_rdl.send(client)
print(boston_train_ptr)
# call create_dataset to create the real Dataset object on remote side
boston_train_ptr.load_dataset()
# call create_dataloader to create the real DataLoader object on remote side
boston_train_ptr.create_dataloader()

Let's check the content of the pointer to remote dataloader

In [17]:
for i, b in enumerate(boston_train_ptr):
    # if i == 0:
    #     X, y = b[0], b[1]
    #     print(X)
    #     print(y)
    X, y = b[0], b[1]
    print(i, len(y))

0 16
1 16
2 16
3 16
4 16
5 16
6 5


In [13]:
class ClientModel(sy.Module):
    def __init__(self, torch_ref):
        super(ClientModel, self).__init__(torch_ref=torch_ref)   
        self.fc1 = self.torch_ref.nn.Linear(13, 128)
        self.relu1 = self.torch_ref.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        output = self.relu1(x)
        return output

In [14]:
class ServerModel(sy.Module):
    def __init__(self, torch_ref):
        super(ServerModel, self).__init__(torch_ref=torch_ref)   
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc2(x)
        x = self.relu2(x)
        output = self.fc3(x)
        return output

In [15]:
client_model = ClientModel(torch_ref=remote_torch)
server_model = ServerModel(torch_ref=torch)

In [16]:
epoch = 400
lr = 0.001

optim_client = remote_torch.optim.Adam(params=client_model.parameters(), lr=lr)
optim_server = torch.optim.Adam(params=server_model.parameters(), lr=lr)

NameError: name 'ecg_client_ptr' is not defined